<a href="https://colab.research.google.com/github/eisbetterthanpi/transformer/blob/main/gpt_tut.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://pytorch.org/tutorials/beginner/transformer_tutorial.html
# https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/9cf2d4ead514e661e20d2070c9bf7324/transformer_tutorial.ipynb
%pip install portalocker
%pip install torchdata

In [ ]:
# @title data
import torch
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

train_iter = WikiText2(split='train') # train_iter will be "consumed" by the process of building the vocab
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])



class Datasetme(torch.utils.data.Dataset):
    def __init__(self, raw_data, tokenizer=tokenizer, vocab=vocab):
        self.data = self.data_process(raw_data) # list of int, [2049990]
        self.bptt = 35 # langth of each batch of sentences
        self.ind = torch.arange(0, self.data.size(0) - 1, step=self.bptt)

    def data_process(self, raw_text_iter): # long string
        data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
        return torch.cat(tuple(filter(lambda t: t.numel() > 0, data))) # list of int, [2049990]

    def __len__(self):
        return len(self.data) // self.bptt

    def __getitem__(self, idx):
        i=idx*self.bptt
        data = self.data[i:i+self.bptt]
        target = self.data[i+1:i+self.bptt+1].reshape(-1)
        return data, target

def detoken(tgt_tokens):
    return " ".join(vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

train_iter, val_iter, test_iter = WikiText2() # line by line of wiki  = Valkyria Chronicles III =
batch_size=128
train_iter = Datasetme(train_iter)
val_iter = Datasetme(val_iter)
test_iter = Datasetme(test_iter)

def collate_fn(data):
    x,y=zip(*data)
    # print("collate",len(x),len(y))
    x=torch.stack(list(x), dim=0) # batch_first->dim=0
    # y=torch.stack(list(y)).T.flatten()
    y=torch.stack(list(y))#.flatten()
    # print(y.shape)
    return x, y

train_loader = torch.utils.data.DataLoader(train_iter, batch_size=batch_size, collate_fn=collate_fn, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_iter, batch_size=batch_size, collate_fn=collate_fn, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_iter, batch_size=batch_size, collate_fn=collate_fn, drop_last=True)

for x,y in train_loader:
    print(x.shape,y.shape) # [128, 35], [128, 35]
    [print(detoken(xx)) for xx in x[:5]]
    print("yyyyyyyyyyyyyyyyyyyyy")
    # print(detoken(y[:100]))
    # [print(detoken(y[i,:10])) for i in range(5)]
    [print(detoken(yy)) for yy in y[:5]]
    break



In [ ]:
# @title sentencepiece tokenizer
!pip install sentencepiece
# https://github.com/kutvonenaki/cc100-sentencepiece
!git clone https://github.com/kutvonenaki/cc100-sentencepiece.git

import sentencepiece as spm

# ​cc100_en_vocab_8000.model # cc100_en_vocab_8000.vocab
modelpath="/content/cc100-sentencepiece/trained_tokenizers/cc100_en_vocab_8000.model"
sp = spm.SentencePieceProcessor(model_file=modelpath)
# text="ayo boy whts'upp? ;) 998"
text="yes tokenizers would be peferable, over traditional cat! dog?"
encoded = sp.encode(text)
print(encoded)
decoded = sp.decode(encoded)
print(decoded)


import torch
from torchtext.datasets import WikiText2
train_iter = WikiText2(split='train') # train_iter will be "consumed" by the process of building the vocab


class Datasetme(torch.utils.data.Dataset):
    def __init__(self, raw_data):
        self.data = self.data_process(raw_data) # list of int, [2049990]
        self.bptt = 35 # langth of each batch of sentences
        self.ind = torch.arange(0, self.data.size(0) - 1, step=self.bptt)

    def data_process(self, raw_text_iter): # long string
        return sp.encode(raw_text_iter)
        # data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
        # return torch.cat(tuple(filter(lambda t: t.numel() > 0, data))) # list of int, [2049990]

    def __len__(self):
        return len(self.data) // self.bptt

    def __getitem__(self, idx):
        i=idx*self.bptt
        data = self.data[i:i+self.bptt]
        target = self.data[i+1:i+self.bptt+1].reshape(-1)
        return data, target

train_iter, val_iter, test_iter = WikiText2() # line by line of wiki  = Valkyria Chronicles III =
batch_size=128
train_iter = Datasetme(train_iter)
val_iter = Datasetme(val_iter)
test_iter = Datasetme(test_iter)

def collate_fn(data):
    x,y=zip(*data)
    # print("collate",len(x),len(y))
    x=torch.stack(list(x), dim=0) # batch_first->dim=0
    # y=torch.stack(list(y)).T.flatten()
    y=torch.stack(list(y))#.flatten()
    # print(y.shape)
    return x, y

train_loader = torch.utils.data.DataLoader(train_iter, batch_size=batch_size, collate_fn=collate_fn, drop_last=True)
val_loader = torch.utils.data.DataLoader(val_iter, batch_size=batch_size, collate_fn=collate_fn, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_iter, batch_size=batch_size, collate_fn=collate_fn, drop_last=True)

for x,y in train_loader:
    print(x.shape,y.shape) # [128, 35], [128, 35]
    [print(detoken(xx)) for xx in x[:5]]
    print("yyyyyyyyyyyyyyyyyyyyy")
    # print(detoken(y[:100]))
    # [print(detoken(y[i,:10])) for i in range(5)]
    [print(detoken(yy)) for yy in y[:5]]
    break

def detoken(tgt_tokens):
    return " ".join(sp.decode(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")




In [ ]:
# @title wandb
# https://docs.wandb.ai/quickstart
!pip install wandb
import wandb
wandb.login() # 487a2109e55dce4e13fc70681781de9f50f27be7
run = wandb.init(
    project="transformer_tut",
    config={
        "model": "adam 1e-3",
        "optim": "adam",
        # "learning_rate": 5,
    })


In [ ]:
# @title Transformer Model
import math
import os
import torch
from torch import nn
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout = 0.1, max_len = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x): # x: [seq_len, batch_size, embedding_dim]
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)


class TransformerModel(nn.Module):
    def __init__(self, ntoken, d_model, nhead, d_hid, nlayers, dropout = 0.5):
        super().__init__()
        self.embedding = nn.Embedding(ntoken, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, d_hid, dropout, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, nlayers)
        self.d_model = d_model
        self.linear = nn.Linear(d_model, ntoken)
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1 # ogtorch 0.1 gpt 0.02
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.linear.bias.data.zero_()
        self.linear.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, src_mask = None): # [seq_len, batch_size], [seq_len, seq_len]
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        if src_mask is None: src_mask = nn.Transformer.generate_square_subsequent_mask(src.shape[1]).to(device)
        output = self.transformer_encoder(src, src_mask)
        # print("fwd",output.shape) # float [seq_len, batch_size, d_model]
        output = self.linear(output) # no  log-softmax bec use CrossEntropyLoss which requires the inputs to be unnormalized logits.
        return output # [seq_len, batch_size, ntoken]

ntokens = len(vocab)  # size of vocabulary ; vocab size is equal to the length of the vocab object
# model = TransformerModel(ntokens, d_model=512, nhead=8, d_hid=512, nlayers=6, dropout=0.1).to(device) # small
model = TransformerModel(ntokens, d_model=512, nhead=8, d_hid=2048, nlayers=8, dropout=0.1).to(device) # mid

# nhead, d_model, nlayers = 12,768,12
# pw_ff 3072 d_hid
# https://pytorch.org/hub/huggingface_pytorch-transformers/
# gpt paper https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf
# bert paper https://arxiv.org/pdf/1810.04805.pdf
# https://vitalflux.com/bert-vs-gpt-differences-real-life-examples/
# Toronto BookCorpus (800M words) and English Wikipedia (2,500M words), BookCorpus

# https://www.analyticsvidhya.com/blog/2022/10/generative-pre-training-gpt-for-natural-language-understanding/


In [6]:
# @title train eval
import torch
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True
scaler = torch.cuda.amp.GradScaler()

# train function with automatic mixed precision
def strain(model, dataloader, optimizer, loss_fn, scheduler=None):
    model.train()
    total_loss = 0.
    for batch, (data, targets) in enumerate(dataloader):
        data, targets = data.to(device), targets.to(device)
        with torch.cuda.amp.autocast(): # automatic mixed percision
            output = model(data) # [batch_size, seq_len, ntoken]
            # print("strain",output.shape,targets.shape) # [128, 35, 28782]) torch.Size([128, 35]
            output = output.view(-1, ntokens)
            targets = targets.reshape(-1)
            # print("strain",output.shape, targets.shape)
            loss = loss_fn(output, targets)
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        scaler.step(optimizer)
        scaler.update()
        # if scheduler is not None: scheduler.step()
        # print("strain",loss.item())
        total_loss += loss.item()
        try: wandb.log({"train loss": loss.item()/len(targets)})
        except NameError: pass
    return total_loss / len(dataloader)


def train(model, dataloader, optimizer, loss_fn):
    model.train()
    total_loss = 0.
    for batch, (data, targets) in enumerate(dataloader):
        data, targets = data.to(device), targets.to(device)
        output = model(data)
        output = output.view(-1, ntokens)
        targets = targets.reshape(-1)
        loss = loss_fn(output, targets)
        loss = loss_fn(output.reshape(-1, output.shape[-1]), targets.reshape(-1)) # batch_first=F -> trg[1:, :]
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(dataloader)

def test(model, loader, loss_fn):
    model.eval()
    total_loss = 0.
    with torch.no_grad():
        for data, targets in loader:
            data, targets = data.to(device), targets.to(device)
            seq_len = data.size(1)
            output = model(data)
            output = output.view(-1, ntokens)
            targets = targets.reshape(-1)
            total_loss += seq_len * loss_fn(output, targets).item()
    return total_loss / len(loader)


In [13]:
# @title generate
def generate(model, src_sentence):
    model.eval()
    src = src_sentence.view(1,-1).to(device)
    num_tokens = src.shape[1]
    trg_indexes = src
    max_len = src.shape[1]+5
    for i in range(max_len):
        with torch.no_grad():
            output = model(trg_indexes)
        pred_token = output.argmax(2)[:,-1].unsqueeze(1)
        trg_indexes=torch.cat((trg_indexes,pred_token),1)
    trg_tokens = trg_indexes.flatten()
    return trg_tokens

def data_process(raw_text_iter):
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))
def detoken(tgt_tokens):
    return " ".join(vocab.lookup_tokens(list(tgt_tokens.cpu().numpy()))).replace("<bos>", "").replace("<eos>", "")

print(detoken(generate(model, data_process(["When he saw the penguin, "]))))


when he saw the penguin , he was not a <unk> . he was also a <unk>


In [ ]:
# @title wwwwwwwwwwwww
import time
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=1e-5, betas=(0.9, 0.98), eps=1e-9) # lr=0.0001 gpt 2.5e-4
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1) # 5. , 0.001
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

for epoch in range(50):
    start = time.time()
    # train_loss = strain(model, train_loader, optimizer, loss_fn, scheduler)
    train_loss = strain(model, train_loader, optimizer, loss_fn)
    # train_loss = train(model, train_loader, optimizer, loss_fn)
    val_loss = test(model, val_loader, loss_fn)
    elapsed = time.time() - start
    print(f'{epoch+1:3d} train loss: {train_loss:5.2f}, valid loss: {val_loss:5.2f}, time: {elapsed:5.2f}s')
    print(detoken(generate(model, data_process(["which led him to ponder the meaning of life, which came to conclude is "]))))
    print(detoken(generate(model, data_process(["When I "]))))
    # scheduler.step()
    print("lr: ", optimizer.param_groups[0]['lr'])

# 1e-1 20 train loss:  5.75, valid loss: 196.86, time: 42.24s
# sgd 1e-1 20 train loss:  5.83, valid loss: 198.11, time: 41.16s
# mid sgd 1e-1 20 train loss:  5.96, valid loss: 202.00, time: 57.21s
#  40 train loss:  5.41, valid loss: 190.40, time: 57.44s



  1 train loss:  5.18, valid loss: 187.03, time: 57.61s
which led him to <unk> the meaning of life , which came to conclude is a <unk> . the <unk> is a <unk> , and a <unk> <unk> <unk> , which is a <unk> .
when i think i think i think i think
lr:  0.1
  2 train loss:  5.16, valid loss: 186.84, time: 57.69s
which led him to <unk> the meaning of life , which came to conclude is a <unk> . the <unk> is a <unk> , and a <unk> <unk> , which is a <unk> . the
when i think i think i think i think
lr:  0.1
  3 train loss:  5.14, valid loss: 186.63, time: 57.24s
which led him to <unk> the meaning of life , which came to conclude is a <unk> . the <unk> is a <unk> , and a <unk> <unk> , which is a <unk> . the
when i think i think i think i '
lr:  0.1
  4 train loss:  5.12, valid loss: 186.32, time: 57.51s
which led him to <unk> the meaning of life , which came to conclude is a <unk> . the <unk> is a <unk> , a <unk> , a <unk> , <unk> , and a <unk>
when i think i think i have a lot
lr:  0.1
  5 train lo

In [17]:
# @title save
from google.colab import drive
drive.mount('/content/gdrive')
PATH="/content/gdrive/MyDrive/torch_save/" # for saving to google drive
name='transformer_tut_mid.pth'
# PATH="/content/" # for saving on colab only
# name='model.pth'

torch.save(model.state_dict(), PATH+name)

# model.load_state_dict(torch.load(PATH+name))
# model.load_state_dict(torch.load(PATH+name, map_location=device))


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
